In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd drive/My Drive

In [0]:
! pip install pillow

import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

import random
import os

import cv2
from PIL import Image

import sys
import shutil
from pathlib import Path
import os.path,sys

In [0]:
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, GlobalAveragePooling2D,Input
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Activation, Dropout, Flatten, Dense
import keras.callbacks
from keras import optimizers


### 前処理

In [0]:
p = Path('.')
classes=['髙橋海人','岸優太','佐藤勝利','中島健斗','横尾渉','宮田俊哉','アンガールズ田中','アンガールズ山根']
classes_num =len(classes)
img_width,img_height=224,224

### データ分割

In [0]:
for word in classes:
    # 元画像ディレクトリのパスオブジェクト
    raw_dir = p  / word
   
    # 訓練データを格納するディレクトリ作成
    train_set = p / 'train_set' / word
    train_set.mkdir(parents=True, exist_ok=True)
    
    # テストデータを格納するディレクトリ作成
    test_set = p / 'test_set' / word
    test_set.mkdir(parents=True, exist_ok=True)
    
     # 各ブランドのの画像一覧取得
     #iterdir()=ディレクトリ内のサブディレクトリorファイル一覧を取得
    imgs_path = list(raw_dir.iterdir())
    random.shuffle(imgs_path)
    # 訓練データを格納
    for img in imgs_path[:int(len(imgs_path)*0.3)]:
        #　テスト画像をコピー
        shutil.copy(str(img), str(test_set.resolve()))
          #str(test_set.resolve())=test_setに岩橋玄樹∼髙橋海人までありますってこと
     #訓練データを格納
    for img in imgs_path[int(len(imgs_path)*0.3):]:
        # 訓練画像をコピー
        shutil.copy(str(img), str(train_set.resolve()))

In [0]:
#画像が分割できているか確認
train_path = Path('./train_set')
test_path = Path('./test_set')

In [0]:
def image_check(check_path):
  check_image = 0
  for brand in check_path.iterdir():
    for img in brand.iterdir():
      check_image += 1
  return check_image

In [0]:
train_imgs = image_check(train_path)
test_imgs = image_check(test_path)
print('train_image: ', rain_imgs)
print('test_image: ', test_imgs)

In [0]:
"""
train_imgs = 0
test_imgs = 0
#画像が分割できているか確認
for tr_brand in train_path.iterdir():
  for tr_img in tr_brand.iterdir():
    train_imgs += 1

for test_brand in test_path.iterdir():
  for test_img in test_brand.iterdir():
    test_imgs += 1
"""

### データ加工

In [0]:
batch = 112
num_train = 448
num_val = 192
nb_epoch=50

In [0]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    #shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    #vertical_flip=True,
    rescale=1.0 / 255
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255
)

In [0]:
train_generator = train_datagen.flow_from_directory(
    'train_set',
    target_size=(img_width,img_height ),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch,
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
   'test_set',
   target_size=(img_width, img_height),
   color_mode='rgb',
   classes=classes,
   class_mode='categorical',
   batch_size=batch,
   shuffle=True
)

## モデル構築


### ファインチューニング

In [0]:
keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)


## モデル構築

In [0]:
# VGG16のロード。FC層は不要なので include_top=False
input_tensor = Input(shape=(img_width, img_height, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# VGG16の図の緑色の部分（FC層）の作成
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(classes_num, activation='softmax'))

# VGG16とFC層を結合してモデルを作成（完成図が上の図）
vgg_model = Model(input=vgg16.input, output=top_model(vgg16.output))

# VGG16の図の青色の部分は重みを固定（frozen）
for layer in vgg_model.layers[:15]:
    layer.trainable = False

# 多クラス分類を指定
vgg_model.compile(loss='categorical_crossentropy',
          optimizer=optimizers.SGD(lr=0.001,momentum=0.9),
          metrics=['accuracy'])

### 学習

In [0]:
hist =vgg_model.fit_generator(
    train_generator,
    samples_per_epoch=num_train,
    epochs=nb_epoch,
    #verbose=1,
    validation_data=validation_generator,
    nb_val_samples=num_val)

## 評価

### 損失関数評価

In [0]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
import matplotlib.pyplot as plt
plt.plot(train_loss, label='train')
plt.plot(val_loss, label='val')
plt.legend()

### 正解率

In [0]:
train_acc = hist.history['acc']
val_acc = hist.history['val_acc']
import matplotlib.pyplot as plt
plt.plot(train_acc, label='train')
plt.plot(val_acc, label='val')
plt.legend()

## 応用(学習が途中で終わったりしたらこれを実行)

In [0]:
#resultsディレクトリを作成
result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

# 重みを保存
vgg_model.save_weights(os.path.join(result_dir, 'Final.h5'))
# 作成したモデルを保存
vgg_model.save('VGGtake1.h5')

In [0]:
# VGG16のロード。FC層は不要なので include_top=False
input_tensor = Input(shape=(img_width, img_height, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# VGG16の図の緑色の部分（FC層）の作成
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(classes_num, activation='softmax'))

# VGG16とFC層を結合してモデルを作成（完成図が上の図）
vgg_model = Model(input=vgg16.input, output=top_model(vgg16.output))

# VGG16の図の青色の部分は重みを固定（frozen）
for layer in vgg_model.layers[:15]:
    layer.trainable = False

# 多クラス分類を指定
vgg_model.compile(loss='categorical_crossentropy',
          optimizer=optimizers.SGD(lr=0.005,momentum=0.5),
          metrics=['accuracy'])

vgg_model = Model(input=vgg16.input, output=top_model(vgg16.output))

from keras.models import load_model
vgg_model.load_weights('./results/Final.h5')

In [0]:
vgg_model.load('VGGtake1.h5')

## 予測

In [0]:
# テスト用のコード
from keras.preprocessing import image

# 画像を読み込んで予測する
def img_predict(filename):
    # 画像を読み込んで4次元テンソルへ変換
    img = image.load_img(filename, target_size=(img_height, img_width))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    # 学習時にImageDataGeneratorのrescaleで正規化したので同じ処理が必要
    # これを忘れると結果がおかしくなるので注意
    x = x / 255.0   
    #表示
    plt.imshow(img)
    plt.show()
    # 指数表記を禁止にする
    np.set_printoptions(suppress=True)

    #画像の人物を予測    
    pred = vgg_model.predict(x)[0]
    #結果を表示する
    print("髙橋海人':0,'岸優太':1,'佐藤勝利':2,'中島健斗':3,'横尾渉':4,'宮田俊哉':5,'アンガールズ田中':6,'アンガールズ山根':7")
    print( pred*100)

In [0]:
test_name = 'IMG_5755.JPG'
img_predict(test_name)